In [ ]:
import time
print(time.ctime())
%load_ext autoreload
%autoreload 2
%autosave 10


In [ ]:
# %matplotlib inline
# from matplotlib import rcParams
# from matplotlib.lines import lineStyles
# import matplotlib.pyplot as plt

# if True:
#     plt.style.use('./mtl2smv2.stl')



In [ ]:
# We import the necessary packages
from tardis.io.config_reader import Configuration
from tardis.model import Radial1DModel
from tardis.io.atom_data.util import download_atom_data
from astropy import units as u
import matplotlib.pyplot as plt
import copy

# Run TARDIS for STELLA data

### Install

conda install -c conda-forge jupyterlab-mathjax3

Every simulation run requires [atomic data](io/configuration/components/atomic/atomic_data.rst) and a [configuration file](io/configuration/index.rst). 

## Atomic Data

We recommend using the [kurucz_cd23_chianti_H_He.h5](https://dev.azure.com/tardis-sn/TARDIS/_apis/git/repositories/tardis-refdata/items?path=atom_data/kurucz_cd23_chianti_H_He.h5&resolveLfs=true) dataset.

In [ ]:
# from tardis.io.atom_data.util import download_atom_data
# download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
from ipywidgets import IntSlider, Label
IntSlider(description=r'\(\int_0^t f\)')

You can also obtain a copy of the atomic data from the [tardis-refdata](https://github.com/tardis-sn/tardis-refdata/tree/master/atom_data) repository.

## Example Configuration File

The configuration file [tardis_stella.yml](https://github.com/baklanovp/tardis/tree/master/stella/tardis_stella.yml) is used throughout this Quickstart.

In [ ]:
# !wget -q -nc https://raw.githubusercontent.com/baklanovp/tardis/tree/master/stella/tardis_stella.yml

In [ ]:
!cat tardis_stella.yml

In [ ]:
base_config = Configuration.from_yaml("tardis_stella.yml")
shell_config = copy.deepcopy(base_config)

shell_config.supernova.time_explosion = 10 * u.day

# # This line is necessary to indicate we are using a built-in shell structure.
# # Do not change it.
# shell_config.model.structure.type = 'specific'

# shell_config.model.structure.velocity.start = 1000 * u.km/u.s
# shell_config.model.structure.velocity.stop = 5000 * u.km/u.s
# shell_config.model.structure.velocity.num = 20

shell_model = Radial1DModel.from_config(shell_config)

print('velocity:\n', shell_model.velocity)
print('v_inner:\n', shell_model.v_inner)
print('v_outer:\n', shell_model.v_outer)
print('v_middle:\n', shell_model.v_middle)
print('v_boundary_inner:\n', shell_model.v_boundary_inner)
print('v_boundary_outer:\n', shell_model.v_boundary_outer)
print('radius:\n', shell_model.radius)
print('r_inner:\n', shell_model.r_inner)
print('r_outer:\n', shell_model.r_outer)
print('r_middle:\n', shell_model.r_middle)
print('volume:\n', shell_model.volume)

## Running the Simulation

To run the simulation, import the `run_tardis` function and create the `sim` object. 

In [ ]:
from tardis import run_tardis

<div class="alert alert-info">

**Note:**

Get more information about the [progress bars](io/output/progress_bars.rst), [logging configuration](io/optional/logging_configuration.ipynb), and [convergence plots](io/visualization/convergence_plot.ipynb). 
    
</div>

In [ ]:
sim = run_tardis("tardis_stella.yml", 
                 virtual_packet_logging=True,
                 show_convergence_plots=True,
                 export_convergence_plots=True,
                 log_level="INFO")  # WARNING  ERROR

In [ ]:
# pause

## Plotting the Spectrum

Finally, plot the generated spectrum with `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
spectrum = sim.runner.spectrum
spectrum_virtual = sim.runner.spectrum_virtual
spectrum_integrated = sim.runner.spectrum_integrated

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 6.5))

spectrum.plot(label="Normal packets")
spectrum_virtual.plot(label="Virtual packets")
spectrum_integrated.plot(label='Formal integral')

plt.xlim(500, 9000)
plt.title("TARDIS example model spectrum")
plt.xlabel("Wavelength [$\AA$]")
plt.ylabel("Luminosity density [erg/s/$\AA$]")
plt.legend()
plt.show()